<a href="https://colab.research.google.com/github/A-Wpro/ML_Predict_Movie_revenue/blob/main/movieMLpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data set
https://www.kaggle.com/tmdb/tmdb-movie-metadata

WIP

TODO LIST :
-PCA ?
-LABEL ENCODER ? instead of dummuies ? 
-scoring and co (1st try = 0.99 score is to hight)
-find best model (not knn)

In [23]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder# creating instance of one-hot-encoder
from sklearn.metrics import mean_squared_error
from datetime import datetime


# Movie feat :  adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count

In [6]:
from datetime import datetime

# current date and time
now = datetime.now()
print(now)
timestamp = datetime.timestamp(now)
print("timestamp =", timestamp)

2020-12-18 23:08:59.509675
timestamp = 1608332939.509675


In [15]:
movies_metadata=pd.read_csv('/content/drive/My Drive/movies_metadata.csv', sep=',')


movies_metadataWithoutUselessFeat = movies_metadata
movies_metadataWithoutUselessFeat = movies_metadataWithoutUselessFeat.drop(movies_metadataWithoutUselessFeat.columns[[1,3, 4,5,6,8,9,11,12,13,17,18, 19,20,21]], axis=1)


movies_metadataWithoutNaN = movies_metadataWithoutUselessFeat.dropna()
print("\nShape de la frame :\n", movies_metadataWithoutNaN.shape) # on  perd 1500 lignes au lieu de 40k en enlevant 3 feats

#CLEANING DATA : TIMESTAMP TO UNIX SECONDS


tab = list()
for time in movies_metadataWithoutNaN.release_date:
    time = time + "00:00:00.000000"
    time = datetime.timestamp(now)
    tab.append(time)
movies_metadataWithoutNaN.release_date = tab
movies_metadataWithoutNaN.release_date =movies_metadataWithoutNaN.release_date.astype(int)





Shape de la frame :
 (45119, 9)
1995-12-15
1608332939


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
#les dummies; on regarde deja commetn ca se passe
movies_metadataWithoutNaN=pd.get_dummies(movies_metadataWithoutNaN, columns=['adult','original_language'])#,''])



In [17]:
X = movies_metadataWithoutNaN.drop(5) #13= revenue; if revenue = 0 then no money was win
y = movies_metadataWithoutNaN.iloc[:,5]
y = y.iloc[1:]
print('y :',y.shape)
print('x :',X.shape)
print(X.dtypes)

y : (45118,)
x : (45118, 98)
budget                   object
popularity               object
release_date              int64
revenue                 float64
runtime                 float64
                         ...   
original_language_vi      uint8
original_language_wo      uint8
original_language_xx      uint8
original_language_zh      uint8
original_language_zu      uint8
Length: 98, dtype: object


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, random_state=42)

In [30]:


RegLin = LinearRegression()
RegLin.fit(X_train, y_train)
y_pred = RegLin.predict(X_test)

print("Score : ", RegLin.score(X_test, y_test))

print("MSE" ,  mean_squared_error(y_test, y_pred))
#MSE = 0.00012 => probably overfited :( I will try to clean data


y_pred_test= RegLin.predict()

Score :  0.999897639668479
MSE 0.00035466615620334644
